In [48]:
# importing libraries
import tensorflow as tf
from tensorflow.keras.models import load_model
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
import re
import numpy as np

In [2]:
# Load the saved model
loaded_model = load_model('improved_model_tf')

2024-02-20 16:38:28.874356: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-20 16:38:28.874425: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-20 16:38:28.874429: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-20 16:38:28.874735: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-20 16:38:28.875022: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-02-20 16:38:29.450405: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [38]:
# data inference => for example negative sentiment
data_inf = {'reviews.text':'I am dissapointed, hate it.'}

# insert into dataframe
data_inf = pd.DataFrame([data_inf])
data_inf

,reviews.text
0,"I am dissapointed, hate it."


In [39]:
# column stopwords
stopwords = ['haven', 'only', 'why', 'o', 'his', 'won', 'how', "weren't", 'under', "mustn't", 'nor', 'which', 'himself', "you're", "aren't", 'shouldn', 'below', 'over', 'up', "shan't", 'than', "it's", 'your', 'against', 'down', "shouldn't", "you've", 'what', 'in', 'wasn', 'them', 'that', 'herself', 'not', 'does', 'had', 'for', 'during', 'further', 'him', 'itself', 'tv', 'isn', "couldn't", 'once', 'one', 'tablet', 'doesn', "wouldn't", 't', 'ourselves', 'before', 'app', 'until', 'while', 'couldn', 'an', 'fire', 'our', 'into', 'about', 'because', 'most', 'who', 'me', 'have', 'themselves', 'these', 'when', 'mustn', 'kid', 'ain', 'and', 'yours', 'do', 'i', "hasn't", 'yourselves', 'has', "she's", 'having', 'to', 've', 'again', 'their', 'some', 'own', 'whom', 'her', 'will', 'we', 'at', 'wouldn', 'other', 'needn', 'or', 'between', 'shan', 'don', 'being', 'd', 'same', 'by', 'with', 'm', 'as', 'from', 'hers', "should've", 's', "you'll", "mightn't", 'all', 'off', 'are', 'very', 'those', 'alexa', 'where', 'didn', 'mightn', 'few', 'but', 'can', 'am', "don't", 'hasn', 'hadn', 'weren', 'out', 'was', 'doing', "you'd", 'on', 'did', "didn't", 'it', "haven't", "needn't", 'no', "isn't", "won't", 'you', 'after', 'then', 'so', 'my', 'she', 'y', 'each', 'such', 'amazon', 'of', 'there', 'been', 'yourself', 'this', 'if', "hadn't", 'be', 'ma', 'just', 'the', 'its', 'more', 're', 'is', 'here', 'kindle', 'any', 'aren', 'echo', 'through', 'ours', 'theirs', 'were', "doesn't", 'now', "that'll", 'should', 'a', 'above', 'they', "wasn't", 'myself', 'both', 'too', 'll', 'he']

# nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/catherinemulyadi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
# function for text preporcessing
def text_preprocessing(filtered_dataset, stopwords):
    # Convert text in 'sentiment.value' column to lowercase
    text = filtered_dataset['reviews.text'].apply(lambda x: x.lower())

    # Mention removal
    text = text.apply(lambda x: re.sub("@[A-Za-z0-9_]+", " ", x))

    # Hashtags removal
    text = text.apply(lambda x: re.sub("#[A-Za-z0-9_]+", " ", x))

    # Newline removal (\n)
    text = text.apply(lambda x: re.sub(r"\\n", " ", x))

    # Whitespace removal
    text = text.apply(lambda x: x.strip())

    # URL removal
    text = text.apply(lambda x: re.sub(r"http\S+", " ", x))
    text = text.apply(lambda x: re.sub(r"www.\S+", " ", x))

    # Non-letter removal (such as emoticon, symbol (like μ, $, 兀), etc
    text = text.apply(lambda x: re.sub("[^A-Za-z\s']", " ", x))

    # Tokenization
    tokens = text.apply(lambda x: word_tokenize(x))

    # Stopwords removal
    tokens = tokens.apply(lambda x: [word for word in x if word not in stopwords])

    # # Stemming
    # stemmer = PorterStemmer()
    # tokens = tokens.apply(lambda x: [stemmer.stem(word) for word in x])

    # Combining Tokens
    text = tokens.apply(lambda x: ' '.join(x))

    return text

In [41]:
# implementasi fungsi
data_inf['reviews.text.processed'] = text_preprocessing(data_inf, stopwords)

# show dataframe
data_inf

,reviews.text,reviews.text.processed
0,"I am dissapointed, hate it.",dissapointed hate


In [42]:
# definisi X
X_inf = data_inf['reviews.text.processed']

In [43]:
# Perform inference using the loaded model
predictions = loaded_model.predict(X_inf)

1/1 [==============================] - 0s 18ms/step


In [55]:
# Assuming the predictions are stored in a NumPy array 'predictions'
prediction_array = np.array(predictions)

# Find the index of the maximum value
max_index = np.argmax(prediction_array)

# Define sentiment labels
sentiment_labels = ["negative", "neutral", "positive"]

# Assign sentiment label based on the index of the maximum value
sentiment = sentiment_labels[max_index]

print("Sentiment value is", sentiment)

Sentiment value is negative
